In [4]:
# Given the new parameters, let's proceed with the calculations in Python.

# Define constants based on the scenario provided
forecasted_sales_units = 32750
average_sales_price_usd = 90000
cost_per_vehicle_eur = 60000
expected_spot_rate = 1.42
forward_rate = 1.45
option_strike = forward_rate
premium = 0.05
mixed_forward = 0.7
mixed_option = 0.7
alternative_forward = 0.5
alternative_option = 1 - alternative_forward

# Define the scenarios and their respective multipliers
scenarios = ['As Forecasted', '30% Below Forecast', '30% Above Forecast']
multipliers = [1, 0.7, 1.3]

# Initialize an empty list to store the results
results = []

# Perform the calculations for each scenario
for scenario, multiplier in zip(scenarios, multipliers):
    # Calculate the total sales and costs
    total_sales_usd = forecasted_sales_units * average_sales_price_usd * multiplier
    total_cost_eur = forecasted_sales_units * cost_per_vehicle_eur * multiplier
    total_cost_usd_at_expected_rate = total_cost_eur * expected_spot_rate
    
    # Calculate the profits for no hedge scenario
    profit_no_hedge = total_sales_usd - total_cost_usd_at_expected_rate
    
    # 100% Hedge with forwards: costs are locked at the forward rate
    total_cost_usd_at_forward_rate = total_cost_eur * forward_rate
    profit_hedge_forwards = total_sales_usd - total_cost_usd_at_forward_rate
    
    # 100% Hedge with options: costs are locked at the option strike price plus the premium
    total_cost_usd_at_option_strike = (total_cost_eur * option_strike) + (forecasted_sales_units * premium)
    profit_hedge_options = total_sales_usd - total_cost_usd_at_option_strike
    
    # Mixed hedge: part of the costs are locked at the forward rate, the rest at the expected rate
    hedged_cost_usd_mixed_forward = total_cost_eur * mixed_forward * forward_rate
    unhedged_cost_usd_mixed_forward = total_cost_eur * (1 - mixed_forward) * expected_spot_rate
    profit_mixed_forward = total_sales_usd - (hedged_cost_usd_mixed_forward + unhedged_cost_usd_mixed_forward)
    
    # Mixed hedge: part of the costs are locked at the option strike price plus the premium, the rest at the expected rate
    hedged_cost_usd_mixed_option = (total_cost_eur * mixed_option * option_strike) + (forecasted_sales_units * mixed_option * premium)
    unhedged_cost_usd_mixed_option = total_cost_eur * (1 - mixed_option) * expected_spot_rate
    profit_mixed_option = total_sales_usd - (hedged_cost_usd_mixed_option + unhedged_cost_usd_mixed_option)
    
    # Alternative hedge: part of the costs are locked at the forward rate, part with options
    hedged_cost_usd_alternative_forward = total_cost_eur * alternative_forward * forward_rate
    hedged_cost_usd_alternative_option = (total_cost_eur * alternative_option * option_strike) + (forecasted_sales_units * alternative_option * premium)
    profit_alternative_hedge = total_sales_usd - (hedged_cost_usd_alternative_forward + hedged_cost_usd_alternative_option)
    
    # Store the results in a dictionary
    results.append({
        'Sales Amount': scenario,
        'No Hedge': profit_no_hedge,
        '100% Hedge with Forwards': profit_hedge_forwards,
        '100% Hedge with Options': profit_hedge_options,
        '70% Hedge with Forwards, 30% No Hedge': profit_mixed_forward,
        '70% Hedge with Options, 30% No Hedge': profit_mixed_option,
        '50% Hedge with Forwards, 50% Hedge with Options': profit_alternative_hedge
    })

# Convert the list of dictionaries to a DataFrame for display
results_df = pd.DataFrame(results).T
results_df


,0,1,2
Sales Amount,As Forecasted,30% Below Forecast,30% Above Forecast
No Hedge,157200000.0,110040000.0,204360000.0
100% Hedge with Forwards,98250000.0,68775000.0,127725000.0
100% Hedge with Options,98248362.5,68773362.5,127723362.5
"70% Hedge with Forwards, 30% No Hedge",115935000.0,81154500.0,150715500.0
"70% Hedge with Options, 30% No Hedge",115933853.75,81153353.75,150714353.75
"50% Hedge with Forwards, 50% Hedge with Options",98249181.25,68774181.25,127724181.25


In [3]:
import pandas as pd

# Constants
forecasted_sales_units = 32750
average_sales_price_usd = 90000
cost_per_vehicle_eur = 60000
expected_spot_rate = 1.42
forward_rate = 1.45
option_strike = forward_rate
premium = 0.05
mixed_forward = 0.7
mixed_option = 0.7
alternative_forward = 0.5
alternative_option = 0.5

# Scenarios and multipliers
scenarios = ['As Forecasted', '30% Below Forecast', '30% Above Forecast']
multipliers = [1, 0.7, 1.3]

# DataFrame initialization
results_df = pd.DataFrame(index=scenarios, columns=['Sales Amount', 'No Hedge', '100% Hedge with Forwards', 
                                                    '100% Hedge with Options', '70% Hedge with Forwards, 30% No Hedge', 
                                                    '70% Hedge with Options, 30% No Hedge',
                                                    '50% Hedge with Forwards, 50% Hedge with Options', 'Finance Cost'])

# Perform calculations
for scenario, multiplier in zip(scenarios, multipliers):
    # Calculate the sales volume and the corresponding revenue and costs
    sales_volume = forecasted_sales_units * multiplier
    results_df.at[scenario, 'Sales Amount'] = sales_volume
    revenue_usd = sales_volume * average_sales_price_usd
    costs_eur = sales_volume * cost_per_vehicle_eur
    
    # No Hedge scenario
    revenue_eur_no_hedge = revenue_usd / expected_spot_rate
    results_df.at[scenario, 'No Hedge'] = revenue_eur_no_hedge - costs_eur
    
    # 100% Hedge with Forwards
    revenue_eur_forward = revenue_usd / forward_rate
    results_df.at[scenario, '100% Hedge with Forwards'] = revenue_eur_forward - costs_eur
    finance_cost_forward = (revenue_usd / expected_spot_rate - revenue_usd / forward_rate) * forward_rate
    
    # 100% Hedge with Options
    # Assuming the options are exercised because the forward rate is used as the strike price
    revenue_eur_option = (revenue_usd / option_strike) - (premium * sales_volume)
    results_df.at[scenario, '100% Hedge with Options'] = revenue_eur_option - costs_eur
    finance_cost_option = premium * sales_volume
    
    # Mixed Hedging strategies
    revenue_eur_mixed_forward = (revenue_usd * mixed_forward / forward_rate) + (revenue_usd * (1 - mixed_forward) / expected_spot_rate)
    revenue_eur_mixed_option = (revenue_usd * mixed_option / option_strike) - (premium * sales_volume * mixed_option) + (revenue_usd * (1 - mixed_option) / expected_spot_rate)
    results_df.at[scenario, '70% Hedge with Forwards, 30% No Hedge'] = revenue_eur_mixed_forward - costs_eur
    results_df.at[scenario, '70% Hedge with Options, 30% No Hedge'] = revenue_eur_mixed_option - costs_eur
    
    # Alternative Hedging
    revenue_eur_alternative = (revenue_usd * alternative_forward / forward_rate) + (revenue_usd * alternative_option / option_strike) - (premium * sales_volume * alternative_option)
    results_df.at[scenario, '50% Hedge with Forwards, 50% Hedge with Options'] = revenue_eur_alternative - costs_eur
    
    # Finance Cost is only relevant for the forward and option scenarios
    results_df.at[scenario, 'Finance Cost'] = -finance_cost_forward if multiplier == 1 else -finance_cost_option

# Convert all numbers in the DataFrame to two decimal places
results_df = results_df.astype(float).round(2).T

results_df


,As Forecasted,30% Below Forecast,30% Above Forecast
Sales Amount,3.275000e+04,22925.00,4.257500e+04
No Hedge,1.107042e+08,77492957.75,1.439155e+08
100% Hedge with Forwards,6.775862e+07,47431034.48,8.808621e+07
100% Hedge with Options,6.775698e+07,47429888.23,8.808408e+07
"70% Hedge with Forwards, 30% No Hedge",8.064230e+07,56449611.46,1.048350e+08
"70% Hedge with Options, 30% No Hedge",8.064116e+07,56448809.09,1.048335e+08
"50% Hedge with Forwards, 50% Hedge with Options",6.775780e+07,47430461.36,8.808514e+07
Finance Cost,-6.227113e+07,-1146.25,-2.128750e+03
